### Notebook for ROI generation from Mouse ABA atlas

In [ ]:
from typing import Dict, List, Tuple
from pathlib import Path
import nibabel as nib
import numpy as np

mouse_template_dir = Path("/opt/animalfmritools/animalfmritools/data_template/MouseABA")
TEMPLATE_LABEL_NIFTI = mouse_template_dir / "P56_Annotation_downsample2.nii.gz"
json_file = mouse_template_dir / "ABA_ontology.json"
roi_dir = mouse_template_dir / "rois"
if not roi_dir.exists():
    roi_dir.mkdir()

Some functions

In [ ]:
def load_json(json_path: Path) -> Dict:
    import json
    
    with open(json_path, 'r') as f:
        data = json.load(f)

    return data

def assert_keys(check_dict: Dict, list_keys: List[str]) -> None:

    for k in check_dict.keys():
        assert k in list_keys

def organize_substructures(aba_dict: Dict, target_key: str, expected_keys: List[str]) -> Dict:
    sub_dict = organize_main_structures(aba_dict)[target_key]['children']
    organized_dict = {item['name']: item for item in sub_dict}
    assert_keys(organized_dict, expected_keys)
    return organized_dict

EXPECTED_KEYS_MAIN = [
    'Basic cell groups and regions',
    'fiber tracts',
    'ventricular systems',
    'grooves',
    'retina'
]

EXPECTED_KEYS_GM = [
    "Cerebrum",
    "Brain stem",
    "Cerebellum",
]

EXPECTED_KEYS_WM = [
    "cranial nerves",
    "cerebellum related fiber tracts",
    "supra-callosal cerebral white matter",
    "lateral forebrain bundle system",
    "extrapyramidal fiber systems",
    "medial forebrain bundle system",
]

EXPECTED_KEYS_VS = [
    "lateral ventricle",
    "interventricular foramen",
    "third ventricle",
    "cerebral aqueduct",
    "fourth ventricle",
    "central canal, spinal cord/medulla",
]

def organize_main_structures(aba_dict: Dict) -> Dict:
    org = aba_dict['msg'][0]['children']
    organized_dict = {item['name']: item for item in org}
    assert_keys(organized_dict, EXPECTED_KEYS_MAIN)
    return organized_dict

def organize_gm_structures(aba_dict: Dict) -> Dict:
    return organize_substructures(aba_dict, 'Basic cell groups and regions', EXPECTED_KEYS_GM)

def organize_wm_structures(aba_dict: Dict) -> Dict:
    return organize_substructures(aba_dict, 'fiber tracts', EXPECTED_KEYS_WM)

def organize_vs_structures(aba_dict: Dict) -> Dict:
    return organize_substructures(aba_dict, 'ventricular systems', EXPECTED_KEYS_VS)

def extract_levels(
    children: Dict, 
    extract_level: int,
    level: int = 1, 
    graph_idxs: Dict = {},
    main_key = None,
    verbose = False,
):

    from copy import deepcopy

    graph_idxs = deepcopy(graph_idxs)
    
    for child in children:
        k_tuple = (child['name'], child['acronym'])
        v_tuple = (child['name'], child['acronym'], child['graph_order'], child['color_hex_triplet'], child['st_level'], child['ontology_id'])
        if extract_level == level:
            graph_idxs[k_tuple] = [v_tuple]
            main_key = k_tuple
            tracker = 'x'
        elif level > extract_level:
            graph_idxs[main_key].append(v_tuple)
            tracker = '>'
        else:
            tracker = 'o'

        if verbose:
            print(f"{tracker} [{str(child['graph_order']).zfill(4)}] {'-'*level} {child['name']} [{child['acronym']}] {level}")
            
        if 'children' in child:
            graph_idxs = extract_levels(child['children'], extract_level, level + 1, graph_idxs, main_key, verbose)

    return graph_idxs

def check_template_for_idx(template_idx: int, template_nifti: str = TEMPLATE_LABEL_NIFTI) -> bool:
    data = nib.load(template_nifti).get_fdata()
    n_voxels_with_idx = np.where(data==template_idx)[0].shape[0]
    if n_voxels_with_idx > 0:
        return True
    else:
        return False

def get_template_coords_from_idx(template_idx: int, template_nifti: str = TEMPLATE_LABEL_NIFTI) -> Tuple:
    data = nib.load(template_nifti).get_fdata()
    coords = np.where(data == template_idx)

    return coords

def save_template_roi(
    parent_structure_label: str,
    roi_label: str,
    template_coords: Tuple, 
    outdir: Path,
    template_nifti: str = TEMPLATE_LABEL_NIFTI
) -> None:

    
    template_img = nib.load(template_nifti)
    template_data = template_img.get_fdata()
    roi_data = np.zeros(template_data.shape)
    roi_data[(template_coords[0,:], template_coords[1,:], template_coords[2,:])] = 1
    roi_img = nib.Nifti1Image(
        roi_data, 
        header = template_img.header,
        affine = template_img.affine,
    )
    output_path = outdir / f"P56_desc-{parent_structure_label}_roi-{roi_label}.nii.gz"
    if not Path(output_path).exists():
        print(f"Saving to {output_path}.")
        nib.save(roi_img, output_path)
    else:
        print(f"{output_path} already exists.")
    
def parse_children(children, level = 1, parent_structure = None, previous_structure = None, outdir: Path = None):
    if len(children) > 0:
        #print('ROI includes: ')
        exist_first = False
        create_roi  = False
        for c in children:
            # Check if nifti label exists in the atlas
            nifti_label = c['graph_order']
            label_exists = check_template_for_idx(nifti_label)
            # Print info
            if label_exists:
                create_roi = True
                #print(f"{'-'*level} [{c['acronym']}] {c['name']} {c['color_hex_triplet']} {label_exists} || PRIOR: {previous_structure}")
                coords = get_template_coords_from_idx(nifti_label)
                if not exist_first:
                    joined_coords = np.vstack(coords)
                    exist_first = True
                else:
                    joined_coords = np.concatenate((coords, joined_coords), axis=1)
            else:
                pass
                #print(f"{'-'*level} [{c['acronym']}] {c['name']} {c['color_hex_triplet']} {label_exists}")
            parse_children(
                c['children'], 
                level = level + 1, 
                parent_structure = parent_structure, 
                previous_structure = c['acronym'], 
                outdir = outdir
            )

        if create_roi:
            #print(">")
            #print(f"SAVING ROI: {previous_structure} - Parent: {parent_structure}")
            #for c in children:
            #    print(f" - {c['name']} [{c['acronym']}]")
            save_template_roi(parent_structure, previous_structure, joined_coords, outdir)
            #print("<|")

Extract annotation info

In [ ]:
aba_onto = load_json(json_file)
main = organize_main_structures(aba_onto)
gm = organize_gm_structures(aba_onto)
wm = organize_wm_structures(aba_onto)
vs = organize_vs_structures(aba_onto)

Create coarser masks of ROIs based on the ontology structure

In [ ]:
for structure in [gm, wm, vs]:
    for structure_ix, (k, v) in enumerate(structure.items()):
        print(f"[{structure_ix + 1}/{len(structure)}] {v['name']} {v['acronym']}")
        parse_children(v['children'], level = 1, parent_structure = v['acronym'], outdir = roi_dir)

Create parent structure ROIs

In [ ]:
descs = {
    "GM": ["CH", "BS", "CB"],
    "WM": ["cm","cbf","lfbs","eps","mfbs",],
    "VS": ["VL", "V4"]
}

for main_structure, desc in descs.items():
    for sub_structure in desc:
        print(main_structure, sub_structure)
        out_path = f"{roi_dir}/P56_desc-{main_structure}_roi-{sub_structure}.nii.gz"
        !fslmerge -t {out_path} {roi_dir}/P56_desc-{sub_structure}_roi-*.nii.gz
        !fslmaths {out_path} -Tmean -bin {out_path}

Create ROIs for preprocessing pipeline

In [ ]:
!fslmaths {roi_dir}/P56_desc-WM_roi-lfbs.nii.gz -add {roi_dir}/P56_desc-WM_roi-mfbs.nii.gz {roi_dir}/pipeline_wm.nii.gz
!cp {roi_dir}/P56_desc-VS_roi-VL.nii.gz {roi_dir}/pipeline_vs.nii.gz
!cp {roi_dir}/P56_desc-GM_roi-CH.nii.gz {roi_dir}/pipeline_gm.nii.gz